## Transfer Learning

1. Fine Tuning - Train the whole model from scartch
2. Feature Extraction - Remove the last classification layer to train it. Don't train the previous parts. 

Helpful tutorial links:
1. https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
2. https://www.learnpytorch.io/06_pytorch_transfer_learning/
3. Model lists: https://pytorch.org/vision/0.8/models.html


In [ ]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision
from torch.utils.data import *
import torch.optim as optim
import torch.nn as nn
from training_utils import *
from torchinfo import summary

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random,argparse, pickle
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support
torch.manual_seed(0)
random.seed(0)

In [ ]:
weights = torchvision.models.AlexNet_Weights.DEFAULT # .DEFAULT = best available weights from pretraining on ImageNet
weights

In [ ]:
# Get the transforms used to create our pretrained weights
auto_transforms = weights.transforms()
auto_transforms

In [ ]:
train_dataset = datasets.CIFAR10('/home/aminul/data/', transform=auto_transforms, train=True, download=True)
test_dataset  = datasets.CIFAR10('/home/aminul/data/', transform=auto_transforms, train=False, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

device = torch.device('cuda')

In [ ]:
weights = torchvision.models.AlexNet_Weights.DEFAULT # .DEFAULT = best available weights 
model = torchvision.models.alexnet(weights=None).to(device)

summary(model=model, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

In [ ]:
# Freeze all base layers in the "features" section of the model (the feature extractor) by setting requires_grad=False
for param in model.features.parameters():
    param.requires_grad = False
    
# Recreate the classifier layer and seed it to the target device
model.classifier = torch.nn.Sequential(
    #torch.nn.Dropout(p=0.2, inplace=True), 
    torch.nn.Linear(in_features=9216, out_features=10,bias=True)
).to(device)

In [ ]:
summary(model=model_new, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

In [ ]:
n_epochs = 5
criterion = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(),lr=0.001,momentum=0.99)

In [ ]:
history = {'train_loss': [], 'test_loss': [],'train_acc':[],'test_acc':[]}

for epoch in range(n_epochs):
    train_loss, train_acc = train(model,train_loader,criterion,optim,device,epoch)
    test_loss, test_acc = test(model,test_loader,criterion,optim,None, None,device,epoch)
    
    history['train_loss'].append(train_loss)
    history['test_loss'].append(test_loss)
    history['train_acc'].append(train_acc)
    history['test_acc'].append(test_acc)

Extract features from model using model.features in this case. Pass the input and you will get the desired features. Now pass this to SVM or other classifiers. 

In [ ]:
model_new = model.features
x,y = next(iter(train_loader))
x,y = x.to(device), y.to(device)
op = model_new(x)
op.shape